In [283]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RandomizedSearchCV, ShuffleSplit, train_test_split

properati = pd.read_csv('datos/caba_para_mapa.csv',error_bad_lines=False)

<IPython.core.display.Javascript object>

In [284]:
properati.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48323 entries, 0 to 48322
Data columns (total 26 columns):
created_on                    48323 non-null object
property_type                 48323 non-null object
place_name                    48323 non-null object
state_name                    48323 non-null object
lat-lon                       48323 non-null object
lat                           48323 non-null float64
lon                           48323 non-null float64
price                         48323 non-null float64
currency                      48323 non-null object
price_aprox_local_currency    48323 non-null float64
price_aprox_usd               48323 non-null float64
surface_total_in_m2           48323 non-null float64
surface_covered_in_m2         48323 non-null float64
price_usd_per_m2              48323 non-null float64
price_per_m2                  48323 non-null float64
floor                         6223 non-null float64
rooms                         33792 non-null float

In [285]:
properati.loc[:,('created_on')] = pd.to_datetime(properati['created_on'])
properati['anio'] = [x.year for x in properati['created_on'] ]

sup_min = 10
sup_max = 150

lat_min = -35
lat_max = -34

lon_min = -59
lon_max = -58

precio_min = 5000
precio_max = 1000000

properati = properati.loc[(properati.price_aprox_usd.notnull())\
                              & (properati.lat.notnull()) & (properati.lon.notnull())\
                        & (properati.price_aprox_usd <= precio_max) & (properati.price_aprox_usd >= precio_min) &\
                         (properati.surface_total_in_m2 <= sup_max) & (properati.surface_total_in_m2   >= sup_min) &\
                      (properati.lat <= lat_max) & (properati.lat >= lat_min) &\
                       (properati.lon <= lon_max) & (properati.lon >= lon_min) & (properati.property_type=="house"),:]

properati = properati.loc[(properati.anio >= 2016), :]

In [286]:
#paso features con strings a numericos
le_barrio = preprocessing.LabelEncoder()
barrios=properati['place_name']
le_barrio.fit(barrios)
properati['place_name'] = le_barrio.transform(barrios)

le_divisa = preprocessing.LabelEncoder()
divisa=properati['currency']
le_divisa.fit(divisa)
properati['currency'] = le_divisa.transform(divisa)

le_tipo = preprocessing.LabelEncoder()
tipos_prop=properati['property_type']
le_tipo.fit(tipos_prop)
properati['property_type'] = le_tipo.transform(tipos_prop)

###  Agrego features

### Pruebo con np.histogram

In [287]:
cant_buckets =50

#buckets
cantidad,rango = np.histogram(properati['price_usd_per_m2'], bins=cant_buckets)
print rango
properati['categories_by_price']=pd.cut(properati['price_per_m2'],rango,labels=np.arange(cant_buckets),include_lowest=True)
properati['price_range']=pd.cut(properati['price_per_m2'],rango)

properati.loc[properati['categories_by_price']!=properati['categories_by_price']]
#properati['categories_by_price']=pd.qcut(x=properati['price_per_m2'],q=cant_buckets,labels=np.arange(cant_buckets))

[  1973.    3678.6   5384.2   7089.8   8795.4  10501.   12206.6  13912.2
  15617.8  17323.4  19029.   20734.6  22440.2  24145.8  25851.4  27557.
  29262.6  30968.2  32673.8  34379.4  36085.   37790.6  39496.2  41201.8
  42907.4  44613.   46318.6  48024.2  49729.8  51435.4  53141.   54846.6
  56552.2  58257.8  59963.4  61669.   63374.6  65080.2  66785.8  68491.4
  70197.   71902.6  73608.2  75313.8  77019.4  78725.   80430.6  82136.2
  83841.8  85547.4  87253. ]


,created_on,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,...,description,title,dist_a_subte,dist_a_tren,dist_a_univ,dist_a_villa,dist_a_zona_anegada,anio,categories_by_price,price_range


In [288]:
#lo casteo a float porque si no tira error 
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 

properati.loc[:,'is_new']=properati.loc[:,'description'].str.lower().str.contains('estrenar')

properati.groupby('categories_by_price').count().sort_values('created_on')

,created_on,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,...,description,title,dist_a_subte,dist_a_tren,dist_a_univ,dist_a_villa,dist_a_zona_anegada,anio,price_range,is_new
categories_by_price,,,,,,,,,,,,,,,,,,,,,
0.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,1
46.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
45.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
43.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
36.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
34.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
33.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
47.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
49.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


# RF Clasificador Casas

In [74]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'],properati['dist_a_tren'],properati['dist_a_villa'],\
        properati['dist_a_zona_anegada'],properati['surface_total_in_m2'],\
        properati['surface_covered_in_m2'],properati['lat'],properati['lon'])
y = properati['categories_by_price']

### Busco hiper-parametros con random search

In [ ]:
%%notify

rf = RandomForestClassifier(n_jobs=-1)

oob_score=[True,False]
criterion = ['gini','entropy']
max_features =["auto","log2",None]
n_estimators = np.arange(10,100,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators,"oob_score":oob_score}

search = RandomizedSearchCV(rf, param_distributions=param_grid ,cv=5,n_iter=5) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

In [50]:
mejor_rf = search.best_estimator_
print mejor_rf.feature_importances_ # la importancia de cada feature (cuanta informacion le provee al rf) ordenado tal y cual esta zipeado el X

[ 0.05884464  0.05478336  0.11352793  0.0660662   0.10702107  0.24419678
  0.14619598  0.12720687  0.08215716]


In [52]:
search.score(X_test,y_test) # pruebo el estimador refiteado con el set de test

0.27767695099818512

# RF Regresor

In [327]:
#preparo set de datos
X = zip(properati['surface_total_in_m2'],\
        properati['surface_covered_in_m2'],properati['lat'],properati['lon'])
y = properati['price_aprox_usd']

In [301]:
%%notify

rf = RandomForestRegressor(n_jobs=-1)

criterion = ['mse','mae']
max_features =["auto","log2",None]
n_estimators = np.arange(10,200,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators}

search = RandomizedSearchCV(rf, param_distributions=param_grid ,cv=5,n_iter=50,scoring='neg_mean_squared_error') #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 166.94 segundos para 50 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: -538942240.144 (std: 61216483.123)
Promedio validation score: -3111828092.909 (std: 466200130.163)
Promedio fit time: 0.262s
Hiper-parametros: {'n_estimators': 50, 'max_features': 'log2', 'criterion': 'mae'}

Puesto: 2
Promedio training score: -558201385.236 (std: 41354277.959)
Promedio validation score: -3131327087.582 (std: 345848701.807)
Promedio fit time: 0.288s
Hiper-parametros: {'n_estimators': 60, 'max_features': 'log2', 'criterion': 'mae'}

Puesto: 3
Promedio training score: -509414611.015 (std: 20978206.495)
Promedio validation score: -3170022707.516 (std: 363480740.331)
Promedio fit time: 0.623s
Hiper-parametros: {'n_estimators': 190, 'max_features': 'log2', 'criterion': 'mae'}

Puesto: 4
Promedio training score: -443560017.863 (std: 21168163.507)
Promedio validation score: -3188353791.615 (std: 338962480.429)
Promedio fit time: 0.457s
Hiper-parametros: {'n_estimators'

<IPython.core.display.Javascript object>

### mae tarda muchos mas que mse y log 2 anda mejor (mejor cv scor 0.76 )

### Hago GS en los mejores

In [ ]:
%%notify

rf = RandomForestRegressor(n_jobs=-1)

criterion = ['mse']
max_features =['log2']
n_estimators = np.arange(1,201,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators}

from sklearn.metrics import r2_score, mean_squared_error, make_scorer

search = GridSearchCV(rf, param_grid=param_grid ,cv=5) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

In [ ]:
#properati['dist_a_tren'],properati['surface_total_in_m2'],properati['surface_covered_in_m2'],properati['lat'],properati['lon'],properati['categories_by_price']
mejor_rf = search.best_estimator_
print mejor_rf.feature_importances_ # la importancia de cada feature (cuanta informacion le provee al rf) ordenado tal y cual esta zipeado el X

In [ ]:
search.score(X_test,y_test)

In [ ]:
errores = mejor_rf.predict(X_test)-y_test
print ("Error maximo:{0}\nError minimo:{1}".format( max(abs(errores)),min(abs(errores))))
print(errores)

### Analsis de errores

In [ ]:
count_max=0
max_error=50
lista=[]
for error in errores:
    if abs(error)>max_error:
        count_max+=1
        lista.append(abs(error))
print count_max
print sorted(lista)

In [ ]:
# the histogram of the data
plt.figure(figsize=(12,8))
plt.hist(errores, 100, facecolor='blue')
plt.xlabel('Errores')
plt.ylabel('Cantidad')
#plt.xlim(-1000, 1000) #para variar el "zoom a 0"
plt.yscale('log')
plt.show()